In [1]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env.yaml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})
os.environ['DATABASE_URL'] = 'sqlite:///:memory:'
os.environ['WSL_DISTRIBUTION'] = 'Ubuntu'

from hemlock import push_app_context

app = push_app_context()

Server initialized for eventlet.


In [24]:
[os.remove(t) for t in app.tmpfiles if os.path.exists(t)]

[None, None, None, None, None, None, None]

In [23]:
from hemlock import Page, Blank, Label

N_FCAST = 5

Page(
    Label(
        '''
        <p>You will now make {} predictions. You will not receive feedback, and these predictions <i>will</i> determine your bonus.</p>

        <p>Additionally, your free trial using the computer model has expired. To continue using the model, we will pair you with another participant to bid for it.</p>

        <p>We will enter both of your bids in a 'second-price auction'. If you outbid the other participant, you will get to keep using the model, but we will deduct the other participant's bid from your bonus. <b>The best strategy is to bid the exact amount you're willing to pay. Trying to 'game the system' by bidding more or less than you're willing to pay will make you worse off.</b></p>

        <p><b>FAQ:</b> Why should I bid exactly what I'm willing to pay? Isn't it usually a good idea to underbid so I pay less?</p>

        <p><b>Answer: No!</b> You're thinking of a 'first-price' auction, not a second-price auction. The difference is that, if you win, you'll pay the <i>other</i> person's bid, not your own. Economists have <a href="https://en.wikipedia.org/wiki/Vickrey_auction#Proof_of_dominance_of_truthful_bidding" target="_blank">mathematically proven</a> that the best thing you can do in a second-price auction is bid exactly as much as you're willing to pay.</p>

        <p><b>Keep reading if you're not convinced. Otherwise, skip to the bottom of the page to enter your bid.</b></p>

        <p>Imagine you're bidding for a house you plan to flip for $100,000. (This means you're willing to pay $100,000 for it). You're wondering whether to bid $80,000 or $100,000. Consider these cases.</p>

        <ol>
            <li>The other person bids $70,000. You get the house for $70,000 whether you bid $80,000 or $100,000.</li>
            <li>The other person bids $110,000. You lose the auction whether you bid $80,000 or $100,000.</li>
            <li>The other person bids $90,000. If you bid $80,000, you lose the auction. If you bid $100,000, you get the house for $90,000 and flip it for a $10,000 profit.</li>
        </ol>

        <p>In general, underbidding (e.g., bidding $80,000 when you're willing to pay $100,000) never makes you better off and sometimes makes you worse off. So you might as well bid the exact amount you're willing to pay.</p>

        <p>The same logic applies to overbidding. Imagine you bid $120,000. The other person bids $110,000. You get the house for $110,000 but flip it for $100,000 at a loss. In general, overbidding (e.g., bidding $120,000 when you're willing to pay at most $100,000) never makes you better off and sometimes makes you worse off. So you might as well bid the exact amount you're willing to pay.</p>
        '''.format(N_FCAST)    
    ),
    Blank(
        ('''
        <p>From previous studies, we estimate that most participants' bonuses will be $0.10 to $1.70 larger if they have the model to assist them.</p>

        <p>How much are you willing to pay (bid) to continue using the model?</p>
        <ul>
            <li><b>I am willing to pay up to $''', ''' to continue using the model</b></li>
            <li><b>I am unwilling to pay more than $''', ''' to continue using the model</b></li>
            <li>This is roughly equivalent to, <b>I expect my bonus to be $''', ''' larger if I continue using the model</b></li>
        </ul>
        '''),
        blank_empty='_____', prepend='$', type='number', step=.01, min=0
    )
).preview()

<Page (transient 139706623560448)>